In [1]:
import os
import glob

def remove_files_with_pattern(folder_path, pattern):
    # Create the full pattern for glob
    full_pattern = os.path.join(folder_path, pattern)
    
    # Use glob to find files matching the pattern
    files_to_remove = glob.glob(full_pattern)
    
    # Remove each file
    for file_path in files_to_remove:
        try:
            os.remove(file_path)
            print(f"Removed: {file_path}")
        except Exception as e:
            print(f"Error removing {file_path}: {e}")

# Example usage
folder_path = 'blob'  # Replace with your folder path
pattern = '*'  # Replace with your desired pattern (e.g., '*.jpg', '*.log', etc.)
remove_files_with_pattern(folder_path, pattern)


Removed: blob/imagen_45.png
Removed: blob/imagen_12.png
Removed: blob/imagen_33.png
Removed: blob/imagen_24.png
Removed: blob/imagen_20.png
Removed: blob/imagen_19.png
Removed: blob/imagen_14.png
Removed: blob/imagen_74.png
Removed: blob/imagen_83.png
Removed: blob/imagen_42.png
Removed: blob/imagen_71.png
Removed: blob/imagen_25.png
Removed: blob/imagen_76.png
Removed: blob/imagen_57.png
Removed: blob/imagen_55.png
Removed: blob/imagen_53.png
Removed: blob/imagen_44.png
Removed: blob/imagen_37.png
Removed: blob/imagen_2.png
Removed: blob/imagen_88.png
Removed: blob/imagen_49.png
Removed: blob/imagen_60.png
Removed: blob/imagen_87.png
Removed: blob/imagen_73.png
Removed: blob/imagen_22.png
Removed: blob/imagen_39.png
Removed: blob/imagen_65.png
Removed: blob/imagen_6.png
Removed: blob/imagen_18.png
Removed: blob/imagen_59.png
Removed: blob/imagen_92.png
Removed: blob/imagen_29.png
Removed: blob/imagen_23.png
Removed: blob/imagen_90.png
Removed: blob/imagen_8.png
Removed: blob/imagen_47

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar datos desde el CSV
df = pd.read_csv('../CPM/output.csv')
df.columns = df.columns.str.strip()
print(df.head())


         t         mx         my    mr       p0x        p0y    p0r        p1x  \
0  0.00000  99.850000  35.000000  0.15  3.960414  54.718405  0.150  43.838350   
1  0.01875  99.839071  35.000000  0.17  3.966836  54.717084  0.162  43.844729   
2  0.03750  99.828148  34.999636  0.17  3.984096  54.713534  0.186  43.861874   
3  0.05625  99.798789  34.998639  0.21  4.016306  54.706909  0.222  43.893869   
4  0.07500  99.778406  34.997936  0.19  4.067314  54.696414  0.270  43.944536   

         p1y    p1r  ...   p26r       p27x       p27y   p27r       p28x  \
0  48.290226  0.150  ...  0.150  66.006185  21.817479  0.150  33.747393   
1  48.288712  0.162  ...  0.162  66.012294  21.819858  0.162  33.753949   
2  48.284643  0.186  ...  0.186  66.028714  21.826256  0.186  33.771571   
3  48.277048  0.222  ...  0.222  66.059353  21.838197  0.222  33.804454   
4  48.265014  0.270  ...  0.270  66.107869  21.857122  0.270  33.856531   

        p28y   p28r       p29x      p29y   p29r  
0  35.148995

In [3]:
# Recorrer cada fila del DataFrame
import json

# Opening JSON file
f = open('../config.json',)
 
# returns JSON object as 
# a dictionary
data = json.load(f)
min_r = data["minRadius"]
N = data["blueN"]
resize = 2
for index, row in df.iterrows():
    fig, ax = plt.subplots(figsize=(20,14))
    # Dibujar el círculo azul (mx, my, mr)
    circle = plt.Circle((row['mx'], row['my']), min_r * resize, color='blue')
    circle_dashed = plt.Circle((row['mx'], row['my']), row['mr']* resize, color='blue', fill=False, linestyle='--')
    ax.add_patch(circle)
    ax.add_patch(circle_dashed)

    # Dibujar los puntos rojos (p0x, p0y, ..., p14x, p14y) con su radio correspondiente (p0r, p1r, ..., p14r)
    for i in range(N):
        circle = plt.Circle((row[f'p{i}x'], row[f'p{i}y']), min_r* resize, color='red')
        circle_dashed = plt.Circle((row[f'p{i}x'], row[f'p{i}y']), row[f'p{i}r']* resize, color='red', fill=False, linestyle='--')# Radio ajustado con pnr
        ax.add_patch(circle)
        ax.add_patch(circle_dashed)

    # Configuración de ejes
    ax.set_aspect('equal')
    ax.set_xlim(0, 100)  # Limite en el eje X de 0 a 100
    ax.set_ylim(0, 70)   # Limite en el eje Y de 0 a 70
    # Guardar la imagen con un nombre único (por ejemplo usando el índice)
    time = row['t']
    plt.title(f'time {time:.2f}')
    plt.savefig(f'blob/imagen_{index}.png')
    
    plt.close('all')


In [1]:
import glob
import os
import imageio.v2 as imageio
from tqdm import tqdm

def create_video(input_pattern, output_file, fps=30):
    # Create a list of images
    images = sorted(glob.glob(input_pattern), key=lambda x: float(x.split('_')[-1][:-4]))
    
    # Check if images were found
    if not images:
        print(f"No images found matching the pattern: {input_pattern}")
        return False

    try:
        # Get the first image to determine dimensions
        first_image = imageio.imread(images[0])
        height, width = first_image.shape[:2]

        # Create a writer object
        writer = imageio.get_writer(output_file, fps=fps, macro_block_size=None)

        # Read and write images
        for image_path in tqdm(images, desc="Processing images"):
            image = imageio.imread(image_path)
            writer.append_data(image)

        # Close the writer
        writer.close()
        
        print(f"Video created successfully: {output_file}")
        return True
    except Exception as e:
        print(f"An error occurred while creating the video: {e}")
        return False

# Usage
input_pattern = "blob/imagen_*.png"
output_file = 'animacion.mp4'
success = create_video(input_pattern, output_file)


Processing images: 100%|██████████| 1344/1344 [01:12<00:00, 18.54it/s]


Video created successfully: animacion.mp4


In [2]:
from IPython.display import Video
url = 'animacion.mp4'
Video(url)